# Evaluation de la qualité des données
Le but de ce notebook est d'identifier les problèmes de qualité des données. 

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///data.db', echo=False)
con = engine.connect()
df_patient = pd.read_sql('select * from patient', con=con)
df_pcr = pd.read_sql('select * from test', con=con)
con.close()

### Analyse rapide des données

In [3]:
# quick data check 
df_patient.head()

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
0,221958,matisse,clarke,13.0,rene street,ellenbrook,2527,wa,19710708.0,32.0,08 86018809,westella
1,771155,joshua,elrick,23.0,andrea place,east preston,2074,nsw,19120921.0,34.0,02 97793152,foxdown
2,231932,alice,conboy,35.0,mountain circuit,prospect,2305,nsw,19810905.0,22.0,02 20403934,None
3,465838,sienna,craswell,39.0,cumberlegeicrescent,henty,3620,wa,19840809.0,30.0,02 62832318,jodane
4,359178,joshua,bastiaans,144.0,lowrie street,campbell town,4051,nsw,19340430.0,31.0,03 69359594,None


Le Dataframe df_patient contient les informations personnelles sur les patients. Dans le détail, on y trouve les 12 colonnes suivantes qui sont explicites:  
* patient_id : Un  identifiant qui semble représenter un clé	
* given_name	
* surname	
* street_number
* address_1	
* suburb	
* postcode	
* state	
* date_of_birth	
* age	
* phone_number	
* address_2


In [4]:
df_pcr.head()

,patient_id,pcr
0,653091,N
1,347667,Negative
2,708481,Positive
3,148825,Negative
4,150081,Negative


Le Dataframe df_pcr contient une colonne patient_id et une colonne pcr qui contient les résultats des tests PCR. On note déjà une première incohérence seulement en regardant les 4 premières lignes de données: les résultats ne sont pas notés de façon consistantes, ainsi les test négatifs sont noté N ou Negative. Nous explorerons ce problème complètement plus tard dans l'analyse. 

## Analyse des données de df_patient

In [5]:
df_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   patient_id     20000 non-null  int64  
 1   given_name     19560 non-null  object 
 2   surname        19575 non-null  object 
 3   street_number  19618 non-null  float64
 4   address_1      19204 non-null  object 
 5   suburb         19788 non-null  object 
 6   postcode       19801 non-null  object 
 7   state          18010 non-null  object 
 8   date_of_birth  17989 non-null  float64
 9   age            16003 non-null  float64
 10  phone_number   19081 non-null  object 
 11  address_2      7893 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 1.8+ MB


Il y a 20 000 lignes dans le Dataframe. Et il y a des données manquantes pour toutes les colonnes excepté la colonne patient_id. On remarque que les colonnes postcode et phone_number sont représenté sous forme de chaine des caractères et non sous forme de nombres. Par contre date_of_birth est représenté comme un nombre décimal, comme l'age. 
La cellule suivante donne le nombre de valeurs nulles pour chaque colonnes. 


In [6]:
df_patient.isnull().sum().sort_values(ascending=False)

address_2        12107
age               3997
date_of_birth     2011
state             1990
phone_number       919
address_1          796
given_name         440
surname            425
street_number      382
suburb             212
postcode           199
patient_id           0
dtype: int64

La colonne avec le plus de données manquantes est address_2. C'est compréhensible car ce n'est qu'un complément d'adresse qui n'est pas toujours nécessaire. Par contre les données manquantes sont problématiques pour les 2 colonnes qui suivent dans la liste: age et date_of_birth. D'ailleurs ces 2 colonnes sont redondantes: On peut calculer age à partir de date_of_birth. 
En bas de la liste, on trouve patient_id qui n'a pas de valeurs nulles. 


### Analyse statistique des colonnes numériques

In [7]:
df_patient.describe()

,patient_id,street_number,date_of_birth,age
count,20000.000000,19618.000000,1.798900e+04,16003.000000
mean,553918.938750,98.812468,1.949638e+07,27.534337
std,260234.961111,2481.908686,2.926916e+05,7.807008
min,100005.000000,0.000000,1.290041e+07,0.000000
25%,330194.250000,10.000000,1.924113e+07,24.000000
50%,556327.500000,24.000000,1.950032e+07,29.000000
75%,779419.500000,60.000000,1.975052e+07,33.000000
max,999993.000000,342951.000000,1.999123e+07,92.000000


De la cellule ci-dessus, nous ne pouvons pas trop tirer de conclusion, excepté pour la colonne age. La moyenne pour la colonne age est 27.534337 et la médianne 29. De plus le 75% percentile est 33. Bien que l'age maximal est 92, nous voyons que la majorité des données sont pour des ages autour de 30 ans. Et donc ces données ne sont pas représentative de toute la population. 

## Colonne patient_id
La colonne patient_id semble être une clé. Et de ce fait toutes les valeurs devraient êtres unique pour identifier correctement une personne. Vérifions ce point. 

In [8]:
# Using the option keep=False to show all the duplicated rows. 
# Moreover a row can be duplicated more than once
df_patient[df_patient.duplicated(subset=['patient_id'], keep=False)].sort_values(by=['patient_id'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
12117,109304,zachary,farronato,30.0,outtrim avenue,como,2196,vic,19090801.0,31.0,07 22894061,the reefs
14839,109304,bailey,donaldson,20.0,tardent street,ryde,0812,qld,19580310.0,26.0,07 13479210,None
4386,110207,toby,brock,4.0,merriman crescent,baralaba,3025,nsw,19000424.0,35.0,08 33842007,leitrim
12989,110207,zali,brock,32.0,hedger street,toorak,5038,act,NaN,22.0,08 96818512,None
10184,115791,hannah,clarke,70.0,galmarra street,mayfield,7010,vic,19830828.0,25.0,04 70760611,None
...,...,...,...,...,...,...,...,...,...,...,...,...
10507,990695,erin,braunack,49.0,moondarra street,broken hill,2640,qld,19830122.0,30.0,03 69523317,yuulong
8764,990936,amy,royle,90.0,whittell crescent,coramba,5032,sa,19950326.0,NaN,08 07309295,tewantin plaza
12563,990936,samantha,green,21.0,brierly street,ardrossan,2140,None,19380210.0,29.0,02 51600621,None
2385,994235,trent,stewart-jones,129.0,macfarland crescent,wangaratta,2732,nsw,NaN,NaN,07 98662458,mountview


Ces 403 lignes dupliqués sont vraiment un problème, car on voit que ce sont des personnes différentes. Les duplications ne viennent pas de fautes de frappes dans la saisie du nom, de l'adresse, ...
Regardons le test PCR pour un de ces patient_id dupliqués. 


In [9]:
# Test result for one of the duplicated patient_id 
df_pcr[ df_pcr['patient_id'] == 109304 ]

,patient_id,pcr
1960,109304,N


Le test est négatif pour le patient 109304, mais on n'a aucun élément pour dire s'il s'applique à Mr farronato ou Mr donaldson. Donc la solution sera de supprimer tous les patient_id dupliqués dans le dataframe df_patient. 

### Colonne date_of_birth
La colonne date_of_birth est censé représenter la date de naissance, mais le format utilisé est surprenant: C'est un nombre décimal avec 8 chiffres. Cependant lorsque l'on compare la colonne date_of_birth avec la colonne age, il n'y a aucun lien logique. Les premiers patients trié par date_of_birth sont listés dans la cellule suivante.

In [11]:
df_patient.sort_values(by=['date_of_birth']).head(50)

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
8882,258272,jack,jessup,509.0,stone place,batlow,None,qld,12900410.0,34.0,08 77804659,mayflower retrmnt vlge
11591,986226,lara,white,6.0,collocott crescent,alice springs,5031,None,19000105.0,22.0,04 53214941,bracken leigh
1689,707847,rosa,crook,27.0,wilkins street,cooma,5125,vic,19000106.0,NaN,03 41108108,None
1102,218033,taylah,None,3.0,ern florence crescent,montmorency,5051,vic,19000112.0,NaN,02 83506886,None
11936,726441,timotby,webb,100.0,None,mount colah,3054,vic,19000118.0,NaN,04 40675254,None
18880,400767,timothy,webb,10.0,None,mount colah,3054,vic,19000118.0,NaN,04 40675254,None
13336,981450,rocca,samantha,29.0,jacka crescent,edgecilff,6024,nsw,19000119.0,38.0,08 42055191,australia arcade
14362,161596,samantha,rocca,29.0,jacka crescent,edgecliff,6024,nsw,19000119.0,38.0,08 42055191,australia arcade
15408,847075,None,haupt,46.0,None,devonport,4871,wa,19000120.0,27.0,03 47375443,None
18101,690620,riley,dixon,47.0,investigator street,hillside,4870,sa,19000121.0,NaN,07 63746321,None


La valeur la plus faible est "12900410.0". On s'attendrai à ce que ce soit le patient le plus agé. Or il n'a que 34 ans. 
Il y a aussi de l'incohérence entre le patient 847075 et 161596: le patient 161596 est d'une unité ("19000119.0" contre "19000120.0") plus petite que le patient 847075 mais il est plus agé de 11 ans ! (38 ans contre 27 ans). 
Donc telle quelle nous ne savons pas comment utiliser les informations dans la colonne date_of_birth. Nous pouvons essayer d'analyser un peu plus cette colonne. Logiquement les personnes avec le même date_of_birth devrait avoir le même age. 
 

In [52]:
df_patient[df_patient.duplicated(subset=['date_of_birth', 'age'], keep=False) & 
           df_patient['date_of_birth'].notnull() & df_patient['age'].notnull()
          ].sort_values(by=['age', 'date_of_birth'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
1609,423412,talixsa,brammy,11.0,wtreet james,maryborough,5333,vic,19060917.0,7.0,02 77001738,tourist park
2407,821864,talissa,brammy,11.0,james street,maryborough,5333,vic,19060917.0,7.0,02 77001738,tourist park
582,876665,madrlixe,beadell,117.0,groom street,wyoning,4218,nsw,19230525.0,7.0,08 81656149,None
3550,549916,madeline,beadell,117.0,groom street,wyoming,4218,nsw,19230525.0,7.0,08 81656149,None
3411,792841,None,eglineton,12.0,helemon street,surry hills,3216,qlg,19300427.0,7.0,02 49550430,north road
...,...,...,...,...,...,...,...,...,...,...,...,...
4971,162433,eva,miles,18.0,collings wtreet,eight mile plains,6232,vic,19941210.0,43.0,07 71242229,rosetta village
12288,169409,eva,miles,91.0,collingsstreet,eight mioe aplains,6232,vic,19941210.0,43.0,07 71242229,rosetta village
17936,753197,eva,miles,91.0,collings street,eight mile plains,6232,vic,19941210.0,43.0,07 71242229,rosetta village
19313,167261,lars,blunden,101.0,armfield place,tarragindi,2031,wa,19080704.0,45.0,04 80689941,None


Mais cela ressemble davantage à des personnes dupliquées. Il serait interresssant de regarder de nouveau ce point, une fois que le dataframe sera nettoyé des doublons.  

### Analyse des doublons
Les colonnes restantes (given_name, surname,  ne surname, street_number, suburb, postcode, state, age, phone_number, et address_2) ne semblent pas poser de problèmes de représentation de données. Sont-elles pour autant fiables? Permettent-elles d'identifier une personne? Y-il-a t'il des doublons dues à des fautes de frappe dans la saisie des information? 
La colonne susceptible de plus précisément identifier une personne est phone_number. 
Regardons s'il y a des lignes dupliquées pour cette colonne. 

In [37]:
df_patient[df_patient.duplicated(subset='phone_number', keep=False)].sort_values(by=['phone_number'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
2634,311830,taaila,None,16.0,mcnicoll place,orange,6230,vic,19180603.0,NaN,02 00325977,None
3872,210155,taalia,None,19.0,mcnicollyplace,orange,6230,vic,19180603.0,64.0,02 00325977,None
16605,525466,taalia,None,19.0,mcnicoll place,orange,6230,vic,19180603.0,26.0,02 00325977,None
248,870746,tyler,able,15.0,marcus clarke street,wannissa,3333,nsw,19830504.0,NaN,02 00493853,None
4986,268616,tyler,able,154.0,marcus clarke street,wanniassa,3333,nsw,19830504.0,NaN,02 00493853,None
...,...,...,...,...,...,...,...,...,...,...,...,...
19899,668176,holly,bristow,35.0,ebden street,dangar island,5024,vic,19501102.0,33.0,None,rocky flat
19901,629087,None,stewart-jones,20.0,jonsson court,aspley,3215,nsw,19490912.0,36.0,None,None
19922,563445,rachel,shepherdson,9.0,calder crescent,junee,6163,None,19151122.0,31.0,None,fernlea
19932,859893,ruby,nascivera,35.0,heard street,roselands,4814,nsw,19841221.0,36.0,None,sheep station


On voit qu'il y 2771 lignes dupliquées. On note que les 3 premières lignes semblent représenter le même patient. Mais il y a des faute de frappe sur address_1 ou street_number. Le bas de la liste montre également que toutes les valeurs nulles sont considérées comme dupliquées, ce qui n'est pas ces patients qui sont clairement différents. Enlevons les valeurs nulles. 

In [38]:
df_patient[df_patient.duplicated(subset='phone_number', keep=False) &
          df_patient['phone_number'].notnull()].sort_values(by=['phone_number'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
2634,311830,taaila,None,16.0,mcnicoll place,orange,6230,vic,19180603.0,NaN,02 00325977,None
3872,210155,taalia,None,19.0,mcnicollyplace,orange,6230,vic,19180603.0,64.0,02 00325977,None
16605,525466,taalia,None,19.0,mcnicoll place,orange,6230,vic,19180603.0,26.0,02 00325977,None
248,870746,tyler,able,15.0,marcus clarke street,wannissa,3333,nsw,19830504.0,NaN,02 00493853,None
4986,268616,tyler,able,154.0,marcus clarke street,wanniassa,3333,nsw,19830504.0,NaN,02 00493853,None
...,...,...,...,...,...,...,...,...,...,...,...,...
7100,178533,ryab,samakovlis,70.0,ferrier place,kirwan,7301,nsw,NaN,31.0,08 98483679,miami
16577,641889,ryan,samakovlis,70.0,ferrier place,kirwan,7310,nsw,NaN,31.0,08 98483679,miami
5195,912481,kiera,feast,5.0,bateman street,moe,5125,nsw,NaN,33.0,08 99975497,workmans cottage
18207,570917,kierl,feqsa,5.0,bateman street,moe,5125,nsw,NaN,33.0,08 99975497,workmans cottage


Nous avons maintenant 1852 lignes dupliquées. Cet extrait nous montre aussi que les fautes de frappe peuvent apparaître dans n'importe quelle colonne: given_name, surname, ...



In [42]:
df_patient[df_patient.duplicated(subset=['given_name','surname', 'address_1'], keep=False) &
          df_patient[['given_name','surname', 'address_1']].notnull().any(axis=1)].sort_values(
          by=['given_name','surname', 'address_1'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
16951,592646,abbey,cadman,5.0,mugga way,burpengary,2554,None,19920502.0,29.0,03 45996223,None
17258,593406,abbey,cadman,5.0,mugga way,burpengary,2545,None,19920502.0,29.0,03 45969923,None
2781,959414,abbey,cropper,17.0,bowden crescent,ringwood,6237,qld,19200312.0,29.0,02 90211615,pangani
5089,560115,abbey,cropper,16.0,bowden crescent,ringwood,6231,qld,19200312.0,21.0,02 90211615,pangani
6627,961858,abbey,green,5.0,wilkins street,ballina,6110,None,19040209.0,7.0,08 75306674,None
...,...,...,...,...,...,...,...,...,...,...,...,...
16552,279300,None,polak,35.0,jess place,annandale,2250,qld,19531130.0,NaN,03 33180608,bonnie doon
1775,310017,None,wehr,57.0,christmas street,mount victoria,3068,nsw,19820227.0,33.0,03 15451529,None
9140,847320,None,wehr,57.0,christmas street,mount victoria,3068,None,19829327.0,33.0,03 15451529,None
7968,215375,None,zimmermann,447.0,birdwood street,beerburrum,4221,vic,19724011.0,29.0,03 10366613,wildefell


L'exemple ci-dessus montre la difficulté de détecter les doublons lorsque l'on ne peut pas s'appuyer sur un idetificateur fort comme phone_number: On a besoin de 3 colonnes ('given_name','surname', 'address_1') pour commencer à identifier un patient. Et l'exemple ci-dessus ne tient pas contre du fait qu'il peut y avoir des fautes de frappe sur ces trois colonnes. 
On voit également qu'il peut y avoir des fautes de frappe sur phone_number.
On note également que la colonne date_of_birth semble être un identificateur de patient, même s'il nous n'avons pas la clé pour récupérer la date de naissance. Donc cette colonne sera utile pour la suppression des doublons. 

In [45]:
df_patient[df_patient.duplicated(subset=['date_of_birth','surname'], keep=False) &
          df_patient[['date_of_birth']].notnull().any(axis=1)].sort_values(
          by=['date_of_birth','surname'])

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
11936,726441,timotby,webb,100.0,None,mount colah,3054,vic,19000118.0,NaN,04 40675254,None
18880,400767,timothy,webb,10.0,None,mount colah,3054,vic,19000118.0,NaN,04 40675254,None
561,814998,tahlia,coleman,24.0,furey street,boondall,3163,nsw,19000310.0,24.0,02 02079822,None
7101,340011,tahlia,coleman,24.0,furey street,boondall,3163,nsw,19000310.0,19.0,02 02079822,None
15771,694055,tahlia,coleman,24.0,furey steet,boondall,3163,nsw,19000310.0,19.0,02 32161703,None
...,...,...,...,...,...,...,...,...,...,...,...,...
16782,193878,amy,mason,12.0,balsillie crescent,view bank,2126,vic,19991025.0,12.0,02 48437211,None
3976,810644,juliana,grosvenor,5.0,connelly pace,3023,port noarlunga south,tas,19991215.0,NaN,03 55227740,None
11283,793044,juliana,grosvenor,5.0,connelly place,port noarlunga south,3023,tas,19991215.0,NaN,03 55227740,None
15581,697565,juliana,grosvenor,2.0,connellt place,port noarlunha south,3023,tas,19991215.0,NaN,03 55227740,None


## Analyse des données de df_pcr
Nous allons d'abord vérifier s'il y a des doublons pour la colonne ptient_id

In [48]:
df_pcr[df_pcr.duplicated(subset=['patient_id'], keep=False)].sort_values(by=['patient_id'])

,patient_id,pcr
169,159500,N
2934,159500,N
8469,169519,Positive
1842,169519,Negative
5618,212217,N
...,...,...
5542,887646,Negative
1025,975512,Negative
2331,975512,N
7477,990695,N


On voit qu'il y a deux cas de figure pour les doublons:  
* Soit la valeur pcr est identique: Il faut dans ce cas juste supprimer le ligne en double 
* Soit les valeur pcr sont différente: Dans ce cas on ne peut pas savoir laquelle est bonne. Il faut donc supprimer les 2 lignes